In [1]:
import numpy as np
import pandas as pd

import os
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

path = '/opt/ml/input/data/train/'

train = pd.read_csv(path + 'train_ratings.csv')

year_data = pd.read_csv(os.path.join(path, 'years.tsv'), sep='\t')
writer_data = pd.read_csv(os.path.join(path, 'writers.tsv'), sep='\t')
title_data = pd.read_csv(os.path.join(path, 'titles.tsv'), sep='\t')
genre_data = pd.read_csv(os.path.join(path, 'genres.tsv'), sep='\t')
director_data = pd.read_csv(os.path.join(path, 'directors.tsv'), sep='\t')

In [49]:
## 가장 인기가 많은 영화 TOP10 추천. BUT 유저가 이미 본 영화는 배제함.

# 새로운 데이터 프레임 만들기.
# [user, item]으로 구성되어있는 데이터 프레임.
# 유저수가 31360 이고 유저마다 10개 아이템을 추천하므로 test은 313600 행을 가짐.
test = pd.DataFrame(columns=['user','item'])

# 유저수와 아이템 수 기록.
num_users = train['user'].nunique()
num_items = train['item'].nunique()

# user_id 모아둔 리스트 만들기.
user_lst = list(train['user'].unique())

# 아이템 id 인기가 많은(시청 기록이 많은) 순으로 기록.
# [2571, 2959, 296 ...]
# 2571 영화가 가장 인기가 많은 영화.
top_items = list(train['item'].value_counts().index)

# 유저마다 시청한 영화 기록 리스트 나오는 판다스(시리즈).
# user_item_lst[user_id] : user_id를 가진 유저가 본 영화 리스트
user_item_lst = train.groupby("user")["item"].apply(list)

# 25분 넘게 걸림..
# 유저에게 가장 인기 있는 상품 추천
# but(가장 핵심.) 유저 기준 이미 본 영화는 추천에서 배제.
# user_id마다 반복문 돌기.
for user in tqdm(user_lst):
    cnt = 0 # 반복문마다 새는 변수.
    # 영화 시청 기록 숫자만큼 반복문 돌기(최대한으로 잡아놓은 것 뿐 이만큼 안돌음)
    # idx : 0, 1, 2, ...
    for idx in range(num_items):
        # 이번 영화가(top_items[idx], 영화 인기 순으로 나옴)
        # 해당 유저가 본 영화(user_item_lst[user])가 아닌 영화라면.
        if top_items[idx] not in user_item_lst[user]:
            cnt += 1 # 해당 유저에게 추천할 영화가 하나 늘었다.
            # 해당 영화(top_items[idx])를 해당 유저에게 추천한다. 즉 데이터 프레임에 user, item을 추가한다.
            test = test.append(pd.DataFrame([[user, top_items[idx]]], columns=['user','item']))
            if cnt == 10: # 한 사람당 추천할 영화는 10개 이므로.
                break

100%|██████████| 31360/31360 [25:16<00:00, 20.69it/s]


In [65]:
# 제출하기 위한 코드.
test.reset_index(drop = True, inplace = True)
test.to_csv("../output/ksy_most_popular.csv", index=False)